In [2]:
!pip install tabula-py
!pip install pdfminer.six
!pip install PyPDF2
!pip install mojimoji
!pip install jeraconv
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from jeraconv import jeraconv
import os
import pandas as pd
import tabula
import warnings
import mojimoji
warnings.simplefilter('ignore')
from datetime import datetime #決算日で使うかも？

In [3]:
!conda env list

# conda environments:
#
base                  *  /opt/anaconda3
django_1                 /opt/anaconda3/envs/django_1
tsr                      /opt/anaconda3/envs/tsr
tsr2                     /opt/anaconda3/envs/tsr2



In [4]:
#業種コード一覧を読み込み
os.chdir('C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_Salesforce格納用データ')
Industry_code = pd.read_excel('TSR業種コード一覧.xlsx')
#業種コードを4桁0埋め
Industry_code['TSR業種コード'] = Industry_code['TSR業種コード'].astype(str).str.zfill(4)
Industry_code['中分類コード'] = Industry_code['中分類コード'].astype(str).str.zfill(2)
Industry_code['小分類コード'] = Industry_code['小分類コード'].astype(str).str.zfill(3)
Industry_code['細分類コード'] = Industry_code['細分類コード'].astype(str).str.zfill(4)
Industry_code = Industry_code.iloc[:,1:]
Industry_code

NameError: name 'os' is not defined

In [243]:
#CSVファイルがあるディレクトリに移動
print(os.getcwd())
path = 'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_TSR元データ\\上場'

C:\Users\miyab\OneDrive - 株式会社M＆Aバリューエンジニアリング\共有用\TSRデータ更新\2023年上期\2023年上期_TSR元データ\上場\上場


In [244]:
#上記path内のディレクトリを取得
dir = os.listdir(path)
dir

['temp', '上場']

In [245]:
#上記ファイル名のパスを取得
path_ = []
for i in dir:
    pathdir = f'{path}\\{i}'
    path_.append(pathdir)
path_

['C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_TSR元データ\\上場\\temp',
 'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_TSR元データ\\上場\\上場']

In [246]:
dir[0]

'temp'

In [251]:
#空のテーブルを作成
columns = ['企業コード(TSR)','上場/未上場','上場区分','TSR調査年月日','企業名ﾌﾘｶﾞﾅ（半角）','代表者氏名ﾌﾘｶﾞﾅ（半角）','会社名','代表者氏名','代表者氏名(姓)','代表者氏名(名)','郵便番号','都道府県','市区町村','町名・番地','建物名','電話番号','設立年月日','創業年月日','資本金（千円単位）','従業員数','業種1No','業種1','業種2No','業種2','業種3No','業種3',
           '事業概要','営業所・支店住所','役員','仕入先','株主構成','株主・代表取締役一致','第一位株主名称','第一位株主_株式保有割合','第二位株主名称','第二位株主_株式保有割合','第三位株主名称','第三位株主_株式保有割合','取引先','財務情報1_時期','財務情報1_売上(千円)','財務情報1_純利益(千円)','財務情報2_時期','財務情報2_売上(千円)','財務情報2_純利益(千円)','財務情報3_時期','財務情報3_売上(千円)','財務情報3_純利益(千円)',
           '取引銀行','売上伸長率','利益伸長率','TSR備考','代表者住所','代表者生年月日','出身地','出身校']
df_ = pd.DataFrame(columns = columns)

#ファイル一式からテキスト化したCSVファイルのみを抽出
# for i in range(0,len(path_)):
for i in range(0,2):
#   try:
    dirlist = []
    filelist = []
    for curdir, dirs, files in os.walk(path_[i]):
        if len(dirs) > 0:
            dirlist += dirs
        if len(files) > 0:
            filelist += files 

    #ファイル名が.*_csv.csvを抽出
    import re
    pattern = '.*_csv.csv'
    filelist_copy = []
    for files in filelist:
        if re.match(pattern,files) :
            csvfile = files
            filelist_copy.append(csvfile)
        else :
            csvfile = ''

    #パスを指定
    os.chdir(f'{path_[i]}')
    print(os.getcwd())
    #ファイル名を指定
    print(filelist_copy)

    #CSVの番号を指定して読み込み
    #フォルダを移動
    for n in range(0,len(filelist_copy)):
        try :
          df_read = pd.read_csv(f'{filelist_copy[n]}',encoding = 'cp932')
        except :
          df_read = pd.read_csv(f'{filelist_copy[n]}',encoding = 'utf-8') 
        df_read = df_read.replace(['Unnamed: 0','Unnamed: 1','Unnamed: 2','named: 0'],'')

        print(f'読み込んだデータ数:{len(df_read)}')
        
        #上場区分を判別するための関数を定義
        def listing(list):
            if list == '未上場':
                listing_division = '未上場'
            else:
                listing_division = '上場'
            return listing_division

        df_read['上場/未上場'] = df_read['上場区分'].apply(lambda x : listing(x))
        
        #住所分割のための関数を定義
        def addres_split(addres):
            import re
            pattern1 = '東京都|北海道|大阪府|京都府|.{2,3}県'
            pattern2 = """'足立区|荒川区|板橋区|江戸川区|大田区|葛飾区|北区|江東区|品川区|渋谷区|新宿区|杉並区|墨田区|世田谷区|台東区|中央区|千代田区|豊島区|中野区|練馬区|文京区|港区|目黒区|
                        .*市.*[^0-9]区|西村山郡|.*?区|市川市|市原市|野々市市|四日市市|廿日市市|.*?市|.*?郡|.*?島'"""
            pattern3 = """.*[- －][0-9 ０-９]{1,4}[^A-Z ^Ａ-Ｚ][A Ａ B Ｂ C Ｃ D Ｄ E Ｅ F Ｆ].*|.*[- －][0-9 ０-９]{1,4}[^A-Z ^Ａ-Ｚ][A Ａ B Ｂ C Ｃ D Ｄ E Ｅ F Ｆ].*|.*[- －][^A-Z ^Ａ-Ｚ][A Ａ B Ｂ C Ｃ D Ｄ E Ｅ F Ｆ].*[0-9 ０-９]{1,4}|.*[- －][^A-Z ^Ａ-Ｚ][A Ａ B Ｂ C Ｃ D Ｄ E Ｅ F Ｆ].*|.*[0-9 ０-９]{1,4}区[0-9 ０-９]{1,4}[- －][0-9 ０-９]{1,4}|.*[0-9 ０-９]{1,4}区[0-9 ０-９]{1,4}|.*[0-9 ０-９]{1,4}番地[0-9 ０-９]{1,4}|[0-9 ０-９]{1,4}区[0-9 ０-９]{1,4}|
                        .*[0-9 ０-９]{1,4}地割.*[0-9 ０-９]{1,6}-[0-9 ０-９]{1,6}-[0-9 ０-９]{1,6}|.*[0-9 ０-９]{1,4}地割.*[0-9 ０-９]{1,6}[- －][0-9 ０-９]{1,6}|.*[0-9 ０-９]{1,4}地割.*[0-9 ０-９]{1,4}|
                        .*[- －][0-9 ０-９]{1,6}|.*[0-9 ０-９]{1,6}条通り[0-9 ０-９]{1,6}|.*[0-9 ０-９]{1,6}条[0-9 ０-９]{1,6}丁目[0-9 ０-９]{1,6}[- －][0-9 ０-９]{1,6}|.*[0-9 ０-９]{1,6}条[0-9 ０-９]{1,6}丁目|.*[0-9 ０-９]{1,6}画地|.*[- －][0-9 ０-９]{1,6}|.*[0-9 ０-９]{1,6}[a-z A-Z]|.*[0-9 ０-９]{1,6}|[一-龥]+"""
            try :
                todohuken = re.match(pattern1,addres).group()
            except :
                todohuken = ''
            addres1 = addres[len(todohuken):]

            try :
                sikuchoson = re.match(pattern2,addres1).group()
            except :
                sikuchoson = ''
            addres2 = addres1[len(sikuchoson):]

            try :
                banchi = re.match(pattern3,addres2).group()
                if re.match('.*?[a-z ａ-ｚ A-Z Ａ-Ｚ].*[ァ-ヴ]{3,}.*',banchi):#建物名にローマ字とカタカナが含まれている場合（Ａｇｏｒａビルディング）
                    banchi = re.sub('[a-z ａ-ｚ A-Z Ａ-Ｚ].*[ァ-ヴ].*','',banchi)
                elif re.search('第[0-9 ０-９ 〇-九].*ビル.*',banchi):#建物名が第～ビルの場合(東興第２ビル２Ｆ)
                    match = re.search('第[0-9 ０-９ 〇-九].*ビル.*',banchi).group() 
                    banchi = banchi.replace(match,'')
                elif re.search('[a-z ａ-ｚ A-Z Ａ-Ｚ]{1,10}[一-龥々ヶノツ]+[ァ-ヴ]*ビル.*',banchi):#建物名にローマ字と漢字が含まれている場合（ＡＬＤＥＺ紗那）
                    match = re.search('[a-z ａ-ｚ A-Z Ａ-Ｚ]{1,10}[一-龥々ヶノツ]+[ァ-ヴ]*ビル.*',banchi).group() 
                    banchi = banchi.replace(match,'')
                elif re.search('[一-龥々ヶノツ]+ビル.*',banchi):#建物名が漢字＋ビルの場合
                    match = re.search('[一-龥々ヶノツ]+ビル.*',banchi).group() 
                    banchi = banchi.replace(match,'')
                elif re.search('[ァ-ヴ - ー]{3,}',banchi):#建物名がカタカナ＋ビルの場合(エスポワールビル)
                    match = re.search('[ァ-ヴ - ー].*',banchi).group() 
                    banchi = banchi.replace(match,'')
                elif re.search('[a-z ａ-ｚ A-Z Ａ-Ｚ]{2,10}[一-龥々ヶノツ]*.*',banchi):#建物名がローマ字＋数字の場合（SAP４０８）
                    match = re.search('[a-z ａ-ｚ A-Z Ａ-Ｚ]{2,10}[一-龥々ヶノツ]*.*',banchi).group()
                    banchi = banchi.replace(match,'')
                elif re.match('.*[a-z ａ-ｚ A-Z Ａ-Ｚ][- －][A-Z Ａ-Ｚ]{1,20}',banchi):#建物名にハイフンが含まれている場合（T-Biz）
                    banchi = re.sub('[a-z ａ-ｚ A-Z Ａ-Ｚ].*','',banchi)
                elif re.match('.*[a-z ａ-ｚ A-Z Ａ-Ｚ][A-Z Ａ-Ｚ]{1,20}',banchi):#建物名がローマ字の場合(Agora)
                    banchi = re.sub('[a-z ａ-ｚ A-Z Ａ-Ｚ].*','',banchi)
                else :
                    banchi = banchi
            except :
                banchi = ''

            addres3 = addres2[len(banchi):]
            try :
                match = re.match('[- －]',addres3).group()
                addres3 = addres3[len(match):]
            except:
                addres3 = addres3
            tatemonomei = addres3

            return todohuken,sikuchoson,banchi,tatemonomei
            todohuken,sikuchoson,banchi,tatemonomei = addres_split(addres)

        #代表者氏名分割のための関数を定義
        import re
        def name_split(name):
          pattern = ' |　|・'
          try :
            sei = re.split(pattern,name)[0]
            mei = re.split(pattern,name)[1]
          except :
            sei = ''
            mei = name
          return sei,mei
          sei,mei = name_split(name)

        #資本金の単位削除のための関数を定義
        def capital_split(capital):
          pattern = ' |　'
          try :
            capital = re.split(pattern,capital)[0]
          except :
            capital = capital.replace('千円','')[0]
          return capital
          # capital = capital_split(capital)

        #売上伸長率'％'削除のための関数を定義
        def percent_replace(sumrate):
          try :
            sumrate = sumrate.replace('%','')
          except :
            sumrate = sumrate
          return sumrate

        #設立、創業年月を日付型に変換するための関数を定義
        def to_timestamp(gengo):
          try :
            time = dt.strptime(gengo,'%Y/%m/%d')
          except :
            time = gengo
          return time

        #不要項目を削除
        df_read = df_read.replace(['Unnamed: 0','Unnamed: 1','Unnamed: 2','named: 0'],'')

        #住所分割＆文字列処理
        df_read['会社名'] = df_read['会社名'].str.replace('\(株\)','株式会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\（株\）','株式会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\(有\)','有限会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\（有\）','有限会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\(名\)','有限会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\（名\）','有限会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\(資\)','有限会社')
        df_read['会社名'] = df_read['会社名'].str.replace('\（資\）','有限会社')
        df_read['会社名'] = df_read['会社名'].apply(mojimoji.han_to_zen)
        df_read['会社名'].str.replace('－','-')
        
        df_read['代表者氏名'] = df_read['代表者氏名'].str.replace(' ','　')
        df_read['代表者氏名(姓)'] = df_read['代表者氏名'].apply(lambda x:name_split(x)[0])
        df_read['代表者氏名(名)'] = df_read['代表者氏名'].apply(lambda x:name_split(x)[1])
        
        #所在地を全角変換
        df_read['所在地'] = df_read['所在地'].apply(mojimoji.han_to_zen)
        #所在地を都道府県、市区町村、町名・番地、建物名に分割
        df_read['都道府県'] = df_read['所在地'].apply(lambda x:addres_split(x)[0])
        df_read['市区町村'] = df_read['所在地'].apply(lambda x:addres_split(x)[1])
        df_read['町名・番地'] = df_read['所在地'].apply(lambda x:addres_split(x)[2])
        df_read['町名・番地'] = df_read['町名・番地'].str.replace('－','-')
        df_read['建物名'] = df_read['所在地'].apply(lambda x:addres_split(x)[3])
        df_read['建物名'] = df_read['建物名'].str.replace('－','-')
        
        #業種1～3に分類
        df_read['業種1No'] = df_read['業種1'].str[:5].astype(str).str.zfill(4)
        df_read['業種1'] = df_read['業種1'].str[5:]
        df_read['業種2No'] = df_read['業種2'].str[:5]
        df_read['業種2'] = df_read['業種2'].str[5:]
        df_read['業種3No'] = df_read['業種3'].str[:5]
        df_read['業種3'] = df_read['業種3'].str[5:]
        #資本金の単位（千円）を削除
        df_read['資本金（千円単位）'] = df_read['資本金（千円単位）'].astype(str).apply(lambda x:capital_split(x))
#        df_read['資本金（千円単位）'].astype(int)
        
        #従業員数の単位（人）を削除
        df_read['従業員数'] = df_read['従業員数'].str.split(' ').str.get(0)
        #伸長率の単位（％）を削除
        df_read['売上伸長率'] = df_read['売上伸長率'].apply(lambda x:percent_replace(x))
        df_read['利益伸長率'] = df_read['利益伸長率'].apply(lambda x:percent_replace(x))
        
        #事業概要を全角変換
        df_read['営業種目'] = df_read['営業種目'].apply(mojimoji.han_to_zen)
        #営業所・支店住所を全角変換
        df_read['営業所・支店住所'] = df_read['営業所・支店住所'].apply(mojimoji.han_to_zen)
        #仕入先を全角変換
        df_read['仕入先'] = df_read['仕入先'].fillna('').apply(mojimoji.han_to_zen)
        #販売先（取引先）を全角変換
        df_read['販売先'] = df_read['販売先'].apply(mojimoji.han_to_zen)
        #株主構成割合の数値を全角変換
        df_read['株主構成'] = df_read['株主構成'].apply(mojimoji.han_to_zen)
        
        #株主一覧をリスト化
        holders_list = df_read['株主構成'].to_list()
        holder1_name_list = []
        holder1_percent_list = []
        holder2_name_list = []
        holder2_percent_list = []
        holder3_name_list = []
        holder3_percent_list = []
        
        #株主一覧から「名前」と「保有割合」を抽出
        for i in range(len(holders_list)):
            try :
                holder1 = re.split('[,，]',holders_list[i])[0]
                holder1_name = re.sub('\(.+?\)|\（.+?\）','',holder1)
                holder1_percent = re.findall("(?<=\().+?(?=\))|(?<=\（).+?(?=\）)", holder1)
            except :
                holder1 = ''
                holder1_name = ''
                holder1_percent = ''
            try :
                holder2 = re.split('[,，]',holders_list[i])[1]
                holder2_name = re.sub('\(.+?\)|\（.+?\）','',holder2)
                holder2_percent = re.findall("(?<=\().+?(?=\))|(?<=\（).+?(?=\）)", holder2)
            except :
                holder2 = ''
                holder2_name = ''
                holder2_percent = ''
            try :
                holder3 = re.split('[,，]',holders_list[i])[2]
                holder3_name = re.sub('\(.+?\)|\（.+?\）','',holder3) 
                holder3_percent = re.findall("(?<=\().+?(?=\))|(?<=\（).+?(?=\）)", holder3)
            except :
                holder3 = ''
                holder3_name = ''
                holder3_percent = ''

            holder1_name_list.append(holder1_name)
            holder1_percent_list.append(holder1_percent)

            holder2_name_list.append(holder2_name)
            holder2_percent_list.append(holder2_percent)

            holder3_name_list.append(holder3_name)
            holder3_percent_list.append(holder3_percent)

        #リスト内の保有割合を囲んでいる特殊文字[]を削除
        holder1_percent_lists = []
        holder2_percent_lists = []
        holder3_percent_lists = []
        for i in range(len(df_read)):
            try :
                holder1_percent = holder1_percent_list[i][0]
            except :
                holder1_percent = ''

            try :
                holder2_percent = holder2_percent_list[i][0]
            except :
                holder2_percent = ''

            try :
                holder3_percent = holder3_percent_list[i][0]
            except :
                holder3_percent = ''

            holder1_percent_lists.append(holder1_percent)
            holder2_percent_lists.append(holder2_percent)
            holder3_percent_lists.append(holder3_percent)

        #株主項目にリストの中身を追加
        df_read['第一位株主名称'] = holder1_name_list
        df_read['第一位株主_株式保有割合'] = holder1_percent_lists
        df_read['第二位株主名称'] = holder2_name_list
        df_read['第二位株主_株式保有割合'] = holder2_percent_lists
        df_read['第三位株主名称'] = holder3_name_list
        df_read['第三位株主_株式保有割合'] = holder3_percent_lists
        
        #株式保有割合を全角変換
        df_read['第一位株主_株式保有割合'] = df_read['第一位株主_株式保有割合'].apply(mojimoji.han_to_zen)
        df_read['第二位株主_株式保有割合'] = df_read['第二位株主_株式保有割合'].apply(mojimoji.han_to_zen)
        df_read['第三位株主_株式保有割合'] = df_read['第三位株主_株式保有割合'].apply(mojimoji.han_to_zen)
        
        df_read['第一位株主_株式保有割合'] = df_read['第一位株主_株式保有割合'].astype(str)
        df_read['第二位株主_株式保有割合'] = df_read['第二位株主_株式保有割合'].astype(str)
        df_read['第三位株主_株式保有割合'] = df_read['第三位株主_株式保有割合'].astype(str)
        
        #代表者、株主一致チェック
        holder_check_list = []
        for i in range(0,len(df_read)):
            representative_ = df_read['代表者氏名'][i]
            representative = representative_.replace('　','')
            target = '　'
            idx = representative_.find(target)
            representative_first_name = representative[:idx]
            first_holder = df_read['第一位株主名称'][i]
            holders_ = df_read['株主構成'][i]
            holders = re.sub('\(.+?\)','',holders_)
            if representative == first_holder:
                check = '◎'
            elif re.findall(representative,holders):
                check = '〇'
            elif representative_first_name in holders:
                check = '△'
            else :
                check = '×'
            holder_check_list.append(check)
        df_read['株主・代表取締役一致'] = holder_check_list
        
        #設立年月を日付型に変換にするための関数を定義
        j2w = jeraconv.J2W()
        list_ = []
        list = df_read['設立年月日'].to_list()
        #元号(漢字表記)を西暦に変換
        for i in range(len(list)):
            try :
                gengo = j2w.convert(list[i])
            except :
                gengo = list[i]
            list_.append(str(gengo))

        #年のみの場合、1月に指定
        list2_ = []
        for i in range(len(list_)):
            if re.match('^[0-9]{4}$',list_[i]):
                gengo = f'{list_[i]}/01/01'
            elif re.match('[0-9]{4}/[0-9]{2}',list_[i]):
                gengo = f'{list_[i]}/01'
            elif re.match(u'[一-龥]+',list_[i]):
                gengo = ''
            else :
                gengo = list[i]
            list2_.append(gengo)
        array = pd.Series(data=list2_)
        df_read['設立年月日'] = array 

        #年月を日付型に変換にするための関数を定義
        j2w = jeraconv.J2W()
        list_ = []
        list = df_read['創業年月日'].to_list()
        #元号(漢字表記)を西暦に変換
        for i in range(len(list)):
            try :
                gengo = j2w.convert(list[i])
            except :
                gengo = list[i]
            list_.append(str(gengo))

        #年のみの場合、1月に指定
        list2_ = []
        for i in range(len(list_)):
            if re.match('^[0-9]{4}$',list_[i]):
                gengo = f'{list_[i]}/01/01'
            elif re.match('[0-9]{4}/[0-9]{2}',list_[i]):
                gengo = f'{list_[i]}/01'
            elif re.match(u'[一-龥]+',list_[i]):
                gengo = ''
            else :
                gengo = list[i]
            list2_.append(gengo)
        array = pd.Series(data=list2_)
        df_read['創業年月日'] = array 

        #代表者の生年月日を日付型にするための関数を定義
        import re
        def birthday(date):
            if date == '' :
                date = ''
            elif re.match('[0-9 ０-９]{4}[/ ／][0-9 ０-９]{2}[/ ／][0-9 ０-９]{2}[/ ／]',date):
                date = date
            elif re.match('[0-9 ０-９]{4}[ |　]',date):
                date = f'{date[:4]}/01/01'
            elif re.match('[0-9 ０-９]{4}[/ ／][0-9 ０-９]{2}[ |　]',date):
                date = f'{date[:7]}/01'
            return date
        df_read['生年月日'] = df_read['生年月日'].apply(lambda x:birthday(x))

        #年月を日付型に変換
        try :
          df_read['設立年月日'] = df_read['設立年月日'].apply(lambda x:to_timestamp(x))
        except :
          df_read['設立年月日'] = df_read['設立年月日']
        try :
          df_read['創業年月日'] = df_read['創業年月日'].apply(lambda x:to_timestamp(x))
        except :
          df_read['創業年月日'] = df_read['創業年月日']

        try :
          df_read['財務情報1_時期'] = pd.to_datetime(df_read['財務情報1_時期'],format='%Y/%m')
          df_read['財務情報1_時期'] = df_read['財務情報1_時期'] + pd.to_timedelta(df_read['財務情報1_時期'].dt.days_in_month,'d') - pd.to_timedelta('1 days')
        except :
          df_read['財務情報1_時期'] = df_read['財務情報1_時期']
#         df_read['財務情報1_売上(千円)'] = df_read['財務情報1_売上(千円)'].astype(int)
#         df_read['財務情報1_純利益(千円)'] = df_read['財務情報1_純利益(千円)'].astype(int)
        
        try :
          df_read['財務情報2_時期'] = pd.to_datetime(df_read['財務情報2_時期'],format='%Y/%m')
          df_read['財務情報2_時期'] = df_read['財務情報2_時期'] + pd.to_timedelta(df_read['財務情報2_時期'].dt.days_in_month,'d') - pd.to_timedelta('1 days')
        except :
          df_read['財務情報2_時期'] = df_read['財務情報2_時期']
        try :
          df_read['財務情報3_時期'] = pd.to_datetime(df_read['財務情報3_時期'],format='%Y/%m')
          df_read['財務情報3_時期'] = df_read['財務情報3_時期'] + pd.to_timedelta(df_read['財務情報3_時期'].dt.days_in_month,'d') - pd.to_timedelta('1 days')
        except :
          df_read['財務情報3_時期'] = df_read['財務情報3_時期']

        df_read = df_read[['企業コード(TSR)','上場/未上場','上場区分','TSR調査年月日','企業名ﾌﾘｶﾞﾅ（半角）','代表者氏名ﾌﾘｶﾞﾅ（半角）','会社名','代表者氏名','代表者氏名(姓)','代表者氏名(名)','郵便番号','都道府県','市区町村','町名・番地','建物名','電話番号','設立年月日','創業年月日','資本金（千円単位）','従業員数','業種1No','業種1','業種2No','業種2','業種3No','業種3',
                   '営業種目','営業所・支店住所','役員','仕入先','株主構成','株主・代表取締役一致','第一位株主名称','第一位株主_株式保有割合','第二位株主名称','第二位株主_株式保有割合','第三位株主名称','第三位株主_株式保有割合','販売先','財務情報1_時期','財務情報1_売上(千円)','財務情報1_純利益(千円)','財務情報2_時期','財務情報2_売上(千円)','財務情報2_純利益(千円)','財務情報3_時期','財務情報3_売上(千円)','財務情報3_純利益(千円)',
                   '取引銀行','売上伸長率','利益伸長率','TSR備考','代表者住所','生年月日','出身地','出身校']]
        df_read = df_read.rename(columns={'営業種目': '事業概要','販売先':'取引先','生年月日':'代表者生年月日'})
        #ファイル名を変更
        filename = filelist_copy[n].replace('.csv','')
        
        #空のテーブルにデータを追加（全データ50万件を格納）
        df_ = df_.append(df_read,ignore_index = True)
        
        #文字列処理後のテーブルをCSVで保存
        try:
            df_read.to_csv(f'{filename}({len(df_read)})最終版.csv',encoding = 'cp932',index=False)
        except:
            df_read.to_csv(f'{filename}({len(df_read)})最終版(utf-8).csv',encoding = 'utf-8',index=False)
        print(f'{n+1}番目のファイルを書き込みました')
#   except:
    df_read = ''

C:\Users\miyab\OneDrive - 株式会社M＆Aバリューエンジニアリング\共有用\TSRデータ更新\2023年上期\2023年上期_TSR元データ\上場\temp
['全国上場企業_4028件.pdf_csv.csv']
読み込んだデータ数:1
1番目のファイルを書き込みました
C:\Users\miyab\OneDrive - 株式会社M＆Aバリューエンジニアリング\共有用\TSRデータ更新\2023年上期\2023年上期_TSR元データ\上場\上場
['全国上場企業_4028件.pdf_csv.csv']
読み込んだデータ数:1
1番目のファイルを書き込みました


In [209]:
df_

,企業コード(TSR),上場/未上場,上場区分,TSR調査年月日,企業名ﾌﾘｶﾞﾅ（半角）,代表者氏名ﾌﾘｶﾞﾅ（半角）,会社名,代表者氏名,代表者氏名(姓),代表者氏名(名),...,財務情報3_売上(千円),財務情報3_純利益(千円),取引銀行,売上伸長率,利益伸長率,TSR備考,代表者住所,代表者生年月日,出身地,出身校
0,300012667,Jリート,上場,2022/12/16,ｲｵﾝﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｾｷ ﾉﾌﾞｱｷ,イオンリート投資法人,関 延明,関,延明,...,"17,696,599","5,869,347",みずほ 三井住友 三井住友信託 三菱UFJ 農林中央金庫 みずほ信託,101,103,イオンのグループ会社でショッピングモールに投資するリート。2022年7月期末時点での保有資産...,,1964/10/09,,
1,35022302,Jリート,上場,2022/12/26,ｻﾝｹｲﾘｱﾙｴｽﾃ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｵｵﾀ ﾕｳｲﾁ,サンケイリアルエステート投資法人,太田 裕一,太田,裕一,...,"2,176,113","1,014,021",みずほ(本店)三井住友(本店営業部)日本政策投資銀行 みずほ信託(本店営業部)三井住友信託(...,109,106,サンケイビルグループの不動産投資法人。2021年9月に4物件を取得し、事業規模拡大している他...,〒114-0003東京都北区豊島8-27-1フレーシェル王子神谷1号棟601,,,
2,296576433,Jリート,上場,2022/01/07,ﾀﾞｲﾜﾊｳｽﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｱｻﾀﾞ ﾄｼﾊﾙ,大和ハウスリート投資法人,浅田 利春,浅田,利春,...,"29,571,582","11,619,603",三井住友信託(本店営業部)三菱UFJ 三井住友(本店営業部)みずほ 日本政策投資銀行 農林中...,97,95,総合型投資法人である。2021年2月期に続き、2021年8月期も増収増益決算。2022年2月...,,1958/08/29,,
3,138088950,Jリート,上場,2022/05/10,ﾄｳｶｲﾄﾞｳﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｴｶﾞﾜ ﾖｳｲﾁ,東海道リート投資法人,江川 洋一,江川,洋一,...,NaN,NaN,みずほ 静岡,166,259,投資運用法人。スポンサー企業9社との投資運用の業務受託。2021年2月設立。2022年1月期...,〒177-0054東京都練馬区立野町14-30-305,,,
4,296429490,Jリート,上場,2023/03/24,ｲﾁｺﾞｵﾌｲｽﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ﾌｸﾅｶﾞ ﾀｶｱｷ,いちごオフィスリート投資法人,福永 隆明,福永,隆明,...,"7,843,853","3,200,733",三井住友 みずほ 三菱UFJ SBI新生 あおぞら りそな 福岡 日本政策投資銀行 オリック...,90,81,中規模オフィスに特化したポートフォリオを構築。都内を中心に2023年2月末時点で保有物件88...,〒106-0046東京都港区元麻布3-8-43,,,法政大学
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,942020448,プライム,上場,2022/10/24,ﾏﾙﾏｴ,ﾏｴﾀﾞ ﾄｼｶｽﾞ,株式会社マルマエ,前田 俊一,前田,俊一,...,"4,388,522","690,561",鹿児島(出水中央)鹿児島相互信金(出水)商工組合中央金庫(鹿児島)日本政策金融公庫 みずほ(...,160,201,半導体やFPD製造装置向け部品を製造。2022年8月期は半導体向け、太陽電池向け好調で増収増益。,〒899-0211鹿児島県出水市知識町820,1966/11/20,鹿児島県,高等学校
4024,692664076,プライム,上場,2022/09/30,ｵｷﾅﾜﾌｲﾅﾝｼﾔﾙｸﾞﾙ-ﾌﾟ,ﾔﾏｼﾛ ﾏｻﾔｽ,株式会社おきなわフィナンシャルグループ,山城 正保,山城,正保,...,NaN,NaN,沖縄(本店営業部),,,沖縄銀行を含む連結子会社10社で構成されており、銀行業を中心に金融関連の総合サービス事業を手...,〒901-2207沖縄県宜野湾市神山1-13-28,1959/09/23,沖縄県,
4025,959001875,プライム,上場,2023/01/26,ﾘﾕｳｷﾕｳｷﾞﾝｺｳ,ｶﾜｶﾐ ﾔｽｼ,株式会社琉球銀行,川上 康,川上,康,...,"42,490,000","4,009,000",日本銀行,104,322,東証一部上場の金融機関で連結子会社は6社で構成、県内地銀ではトップクラスであり本社建て替え工...,〒902-0061沖縄県那覇市古島2-18-13,1961/08/19,沖縄県,
4026,950017930,プライム,上場,2022/11/17,ｻﾝｴ-,ｼﾝｼﾞﾖｳ ｹﾝﾀﾛｳ,株式会社サンエー,新城 健太郎,新城,健太郎,...,"190,713,000","7,486,000",沖縄(本店営業部)琉球(本店営業部)みずほ(那覇)三菱UFJ信託(本店営業部)農林中央金庫 ...,101,112,沖縄最大手の流通業者でスーパー、コンビニ等を展開。2022年2月期連結は食料品の需要拡大によ...,〒901-0504沖縄県島尻郡八重瀬町後原404-8,1968/11/15,沖縄県,琉球大学


In [210]:
#代表者名のスペースを削除しリスト格納
df_['代表者氏名チェック用'] = df_['代表者氏名'].str.replace(' ','')
df_['代表者氏名チェック用'] = df_['代表者氏名チェック用'].str.replace('　','')
df_['代表者氏名チェック用']

0         関延明
1        太田裕一
2        浅田利春
3        江川洋一
4        福永隆明
        ...  
4023     前田俊一
4024     山城正保
4025      川上康
4026    新城健太郎
4027     本永浩之
Name: 代表者氏名チェック用, Length: 4028, dtype: object

In [211]:
#業種Noから業種分類を特定
df_['業種1No'] = df_['業種1No'].str.strip()
df_['業種2No'] = df_['業種2No'].str.strip()
df_['業種3No'] = df_['業種3No'].str.strip()
df_

,企業コード(TSR),上場/未上場,上場区分,TSR調査年月日,企業名ﾌﾘｶﾞﾅ（半角）,代表者氏名ﾌﾘｶﾞﾅ（半角）,会社名,代表者氏名,代表者氏名(姓),代表者氏名(名),...,財務情報3_純利益(千円),取引銀行,売上伸長率,利益伸長率,TSR備考,代表者住所,代表者生年月日,出身地,出身校,代表者氏名チェック用
0,300012667,Jリート,上場,2022/12/16,ｲｵﾝﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｾｷ ﾉﾌﾞｱｷ,イオンリート投資法人,関 延明,関,延明,...,"5,869,347",みずほ 三井住友 三井住友信託 三菱UFJ 農林中央金庫 みずほ信託,101,103,イオンのグループ会社でショッピングモールに投資するリート。2022年7月期末時点での保有資産...,,1964/10/09,,,関延明
1,35022302,Jリート,上場,2022/12/26,ｻﾝｹｲﾘｱﾙｴｽﾃ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｵｵﾀ ﾕｳｲﾁ,サンケイリアルエステート投資法人,太田 裕一,太田,裕一,...,"1,014,021",みずほ(本店)三井住友(本店営業部)日本政策投資銀行 みずほ信託(本店営業部)三井住友信託(...,109,106,サンケイビルグループの不動産投資法人。2021年9月に4物件を取得し、事業規模拡大している他...,〒114-0003東京都北区豊島8-27-1フレーシェル王子神谷1号棟601,,,,太田裕一
2,296576433,Jリート,上場,2022/01/07,ﾀﾞｲﾜﾊｳｽﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｱｻﾀﾞ ﾄｼﾊﾙ,大和ハウスリート投資法人,浅田 利春,浅田,利春,...,"11,619,603",三井住友信託(本店営業部)三菱UFJ 三井住友(本店営業部)みずほ 日本政策投資銀行 農林中...,97,95,総合型投資法人である。2021年2月期に続き、2021年8月期も増収増益決算。2022年2月...,,1958/08/29,,,浅田利春
3,138088950,Jリート,上場,2022/05/10,ﾄｳｶｲﾄﾞｳﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｴｶﾞﾜ ﾖｳｲﾁ,東海道リート投資法人,江川 洋一,江川,洋一,...,NaN,みずほ 静岡,166,259,投資運用法人。スポンサー企業9社との投資運用の業務受託。2021年2月設立。2022年1月期...,〒177-0054東京都練馬区立野町14-30-305,,,,江川洋一
4,296429490,Jリート,上場,2023/03/24,ｲﾁｺﾞｵﾌｲｽﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ﾌｸﾅｶﾞ ﾀｶｱｷ,いちごオフィスリート投資法人,福永 隆明,福永,隆明,...,"3,200,733",三井住友 みずほ 三菱UFJ SBI新生 あおぞら りそな 福岡 日本政策投資銀行 オリック...,90,81,中規模オフィスに特化したポートフォリオを構築。都内を中心に2023年2月末時点で保有物件88...,〒106-0046東京都港区元麻布3-8-43,,,法政大学,福永隆明
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,942020448,プライム,上場,2022/10/24,ﾏﾙﾏｴ,ﾏｴﾀﾞ ﾄｼｶｽﾞ,株式会社マルマエ,前田 俊一,前田,俊一,...,"690,561",鹿児島(出水中央)鹿児島相互信金(出水)商工組合中央金庫(鹿児島)日本政策金融公庫 みずほ(...,160,201,半導体やFPD製造装置向け部品を製造。2022年8月期は半導体向け、太陽電池向け好調で増収増益。,〒899-0211鹿児島県出水市知識町820,1966/11/20,鹿児島県,高等学校,前田俊一
4024,692664076,プライム,上場,2022/09/30,ｵｷﾅﾜﾌｲﾅﾝｼﾔﾙｸﾞﾙ-ﾌﾟ,ﾔﾏｼﾛ ﾏｻﾔｽ,株式会社おきなわフィナンシャルグループ,山城 正保,山城,正保,...,NaN,沖縄(本店営業部),,,沖縄銀行を含む連結子会社10社で構成されており、銀行業を中心に金融関連の総合サービス事業を手...,〒901-2207沖縄県宜野湾市神山1-13-28,1959/09/23,沖縄県,,山城正保
4025,959001875,プライム,上場,2023/01/26,ﾘﾕｳｷﾕｳｷﾞﾝｺｳ,ｶﾜｶﾐ ﾔｽｼ,株式会社琉球銀行,川上 康,川上,康,...,"4,009,000",日本銀行,104,322,東証一部上場の金融機関で連結子会社は6社で構成、県内地銀ではトップクラスであり本社建て替え工...,〒902-0061沖縄県那覇市古島2-18-13,1961/08/19,沖縄県,,川上康
4026,950017930,プライム,上場,2022/11/17,ｻﾝｴ-,ｼﾝｼﾞﾖｳ ｹﾝﾀﾛｳ,株式会社サンエー,新城 健太郎,新城,健太郎,...,"7,486,000",沖縄(本店営業部)琉球(本店営業部)みずほ(那覇)三菱UFJ信託(本店営業部)農林中央金庫 ...,101,112,沖縄最大手の流通業者でスーパー、コンビニ等を展開。2022年2月期連結は食料品の需要拡大によ...,〒901-0504沖縄県島尻郡八重瀬町後原404-8,1968/11/15,沖縄県,琉球大学,新城健太郎


In [212]:
#業種コード一覧をマージ
df_merge1 = pd.merge(df_,Industry_code,left_on = '業種1No',right_on = '細分類コード',how = 'left')
df_merge2 = pd.merge(df_merge1,Industry_code,left_on = '業種2No',right_on = '細分類コード',how = 'left')
df_merge3 = pd.merge(df_merge2,Industry_code,left_on = '業種3No',right_on = '細分類コード',how = 'left')
df_merge3.iloc[:,56:]
df_merge3.columns
df_merge3 = df_merge3.rename(columns = {'大分類_x':'大分類1','中分類コード_x':'中分類コード1','中分類_x':'中分類1','小分類コード_x':'小分類コード1','小分類_x':'小分類1','細分類コード_x':'細分類コード1', '細分類_x':'細分類1',
       '大分類_y':'大分類2','中分類コード_y':'中分類コード2', '中分類_y':'中分類2', '小分類コード_y':'小分類コード2', '小分類_y':'小分類2','細分類コード_y':'細分類コード2', '細分類_y':'細分類2', 
       '大分類':'大分類3','中分類コード':'中分類コード3', '中分類':'中分類3', '小分類コード':'小分類コード3', '小分類':'小分類3','細分類コード':'細分類コード3', '細分類':'細分類3'})
# df_merge3.to_excel('TSRデータ_業種一覧追加ver.xlsx')

In [252]:
df_merge3

,企業コード(TSR),上場/未上場,上場区分,TSR調査年月日,企業名ﾌﾘｶﾞﾅ（半角）,代表者氏名ﾌﾘｶﾞﾅ（半角）,会社名,代表者氏名,代表者氏名(姓),代表者氏名(名),...,小分類2,細分類コード2,細分類2,大分類3,中分類コード3,中分類3,小分類コード3,小分類3,細分類コード3,細分類3
0,300012667,Jリート,上場,2022/12/16,ｲｵﾝﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｾｷ ﾉﾌﾞｱｷ,イオンリート投資法人,関 延明,関,延明,...,不動産賃貸業,6911,貸事務所業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35022302,Jリート,上場,2022/12/26,ｻﾝｹｲﾘｱﾙｴｽﾃ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｵｵﾀ ﾕｳｲﾁ,サンケイリアルエステート投資法人,太田 裕一,太田,裕一,...,不動産賃貸業,6911,貸事務所業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,296576433,Jリート,上場,2022/01/07,ﾀﾞｲﾜﾊｳｽﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｱｻﾀﾞ ﾄｼﾊﾙ,大和ハウスリート投資法人,浅田 利春,浅田,利春,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,138088950,Jリート,上場,2022/05/10,ﾄｳｶｲﾄﾞｳﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ｴｶﾞﾜ ﾖｳｲﾁ,東海道リート投資法人,江川 洋一,江川,洋一,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,296429490,Jリート,上場,2023/03/24,ｲﾁｺﾞｵﾌｲｽﾘ-ﾄﾄｳｼﾎｳｼﾞﾝ,ﾌｸﾅｶﾞ ﾀｶｱｷ,いちごオフィスリート投資法人,福永 隆明,福永,隆明,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,942020448,プライム,上場,2022/10/24,ﾏﾙﾏｴ,ﾏｴﾀﾞ ﾄｼｶｽﾞ,株式会社マルマエ,前田 俊一,前田,俊一,...,金属素形材製品製造業,2452,金属プレス製品製造業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4024,692664076,プライム,上場,2022/09/30,ｵｷﾅﾜﾌｲﾅﾝｼﾔﾙｸﾞﾙ-ﾌﾟ,ﾔﾏｼﾛ ﾏｻﾔｽ,株式会社おきなわフィナンシャルグループ,山城 正保,山城,正保,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4025,959001875,プライム,上場,2023/01/26,ﾘﾕｳｷﾕｳｷﾞﾝｺｳ,ｶﾜｶﾐ ﾔｽｼ,株式会社琉球銀行,川上 康,川上,康,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4026,950017930,プライム,上場,2022/11/17,ｻﾝｴ-,ｼﾝｼﾞﾖｳ ｹﾝﾀﾛｳ,株式会社サンエー,新城 健太郎,新城,健太郎,...,他の織物・衣服・身の回り品小売業,5799,他の織物・衣服・身の回り品小売業,宿泊業，飲食サービス業,75,宿泊業,751,旅館，ホテル,7511,"旅館,ホテル"


In [214]:
#ディレクトリを変更
os.chdir('C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_Salesforce格納用データ')

### （ⅰ）TSR企業コードでマージ

In [215]:
#salesforceのデータを読み込み
sf = pd.read_csv('Salesforce_TSRコード一覧.csv',encoding ='cp932')
#TSR企業コードがある先に絞る
sf_TSR = sf[sf['企業コード(TSR)'].notnull()]
sf_NoTSR = sf[sf['企業コード(TSR)'].isnull()]

In [216]:
sf_TSR['企業コード(TSR)'] = sf_TSR['企業コード(TSR)'].astype(int)

In [217]:
sf_TSR_ID = sf_TSR[['企業情報 ID','企業コード(TSR)']]
sf_TSR_ID

,企業情報 ID,企業コード(TSR)
1,0015j00000f45G4,140109277
2,0015j00000fpkKm,400396530
3,0015j00000fpkQR,401002004
4,0015j00000f5ZUW,402746465
5,0015j00000fpjqH,512049220
...,...,...
557531,0015j00000fq7e3,650027922
557532,0015j00000fq7e4,650051424
557533,0015j00000fq7ir,650134729
557534,0015j00000fq6xF,641420463


In [218]:
df_update = df_merge3[df_merge3['企業コード(TSR)'].isin(sf_TSR['企業コード(TSR)'])]
print(f'TSR企業コードが合致する件数：{len(df_update)}件')

TSR企業コードが合致する件数：4017件


In [219]:
os.getcwd()

'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_Salesforce格納用データ'

In [220]:
#事務修正ファイルを読み込み（ディレクトリ変更）
os.chdir('2023上期_アップデートファイル')
sf_change = pd.read_csv('会社情報_変更内容.csv',encoding = 'cp932')
sf_change = sf_change.fillna(0)
sf_change['企業コード(TSR)'] = sf_change['企業コード(TSR)'].astype(int)

#事務修正ファイルのうち、TSR企業コードがないもの
sf_change_NoTSR = sf_change[sf_change['企業コード(TSR)'].isnull()]
sf_change['変更内容選択（事務記載）'].value_counts()

会社住所変更                     691
廃業                         241
代表者変更                      118
会社名変更                       56
代表者住所変更                     43
会社電話番号変更                    32
会社住所変更; 代表者変更               24
株主構成変更                      18
設立年月日変更                     13
代表者変更; 代表者住所変更               4
会社住所変更; 設立年月日変更              4
その他                          3
会社名変更; 代表者変更                 2
会社住所変更; 代表者変更; 代表者住所変更       2
代表者変更; 代表者住所変更; その他          2
会社住所変更; 会社電話番号変更; 代表者変更      1
会社名変更; 会社住所変更                1
Name: 変更内容選択（事務記載）, dtype: int64

In [221]:
# #会社情報の変更がないもの
# sf_change_Nocompany = sf_change[~sf_change['変更内容選択（事務記載）'].str.contains('会社名変更')]
# sf_change_Nocompany

### （ⅱ）事務修正有のデータをTSRコードでマージ

In [222]:
#一部項目アップデート
df_update_part = df_update[df_update['企業コード(TSR)'].isin(sf_change['企業コード(TSR)'])]
print(f'df_update_part（{len(df_update_part)}件）："TSR企業コード"をキーに一部項目アップデート')
#TSR格納区分を指定
df_update_part['TSRデータ格納区分'] = '2023年上期アップデート(一部項目)'
df_update_part = df_update_part[['企業コード(TSR)', '上場/未上場','上場区分','TSR調査年月日', '企業名ﾌﾘｶﾞﾅ（半角）', '代表者氏名ﾌﾘｶﾞﾅ（半角）',
       '会社名', '代表者氏名(姓)', '代表者氏名(名)', '郵便番号', '都道府県', '市区町村', '町名・番地',
       '建物名', '電話番号', '設立年月日', '創業年月日', '資本金（千円単位）', '従業員数', '業種1', '業種2',
       '業種3', '事業概要', '営業所・支店住所', '役員', '仕入先', '株主構成','株主・代表取締役一致',
       '第一位株主名称','第一位株主_株式保有割合','第二位株主名称','第二位株主_株式保有割合','第三位株主名称','第三位株主_株式保有割合','取引先',
       '財務情報3_時期','財務情報3_売上(千円)', '財務情報3_純利益(千円)', '財務情報2_時期', '財務情報2_売上(千円)','財務情報2_純利益(千円)', '財務情報1_時期', '財務情報1_売上(千円)', '財務情報1_純利益(千円)',
       '取引銀行','売上伸長率', '利益伸長率', 'TSR備考', '代表者住所', '代表者生年月日', '出身地', '出身校','TSRデータ格納区分']]
df_update_part = pd.merge(sf_TSR_ID,df_update_part,on='企業コード(TSR)')
df_update_part.to_excel('df_update_part.xlsx',encoding='utf-8',index = False)

df_update_part（49件）："TSR企業コード"をキーに一部項目アップデート


In [223]:
#全項目アップデート
df_update_all = df_update[~df_update['企業コード(TSR)'].isin(sf_change['企業コード(TSR)'])]
print(f'df_update_all({len(df_update_all)}件)："TSR企業コード"をキーに全項目アップデート')
#TSR格納区分を指定
df_update_all['TSRデータ格納区分'] = '2023年上期アップデート(全項目)'
df_update_all = df_update_all[['企業コード(TSR)', '上場/未上場','上場区分', 'TSR調査年月日', '企業名ﾌﾘｶﾞﾅ（半角）', '代表者氏名ﾌﾘｶﾞﾅ（半角）',
       '会社名', '代表者氏名(姓)', '代表者氏名(名)', '郵便番号', '都道府県', '市区町村', '町名・番地',
       '建物名', '電話番号', '設立年月日', '創業年月日', '資本金（千円単位）', '従業員数',
       '大分類1', '中分類コード1', '中分類1', '小分類コード1','小分類1', '細分類コード1', '細分類1',
       '大分類2', '中分類コード2', '中分類2', '小分類コード2', '小分類2','細分類コード2', '細分類2',
       '大分類3', '中分類コード3', '中分類3', '小分類コード3', '小分類3','細分類コード3', '細分類3', 
       '事業概要', '営業所・支店住所', '役員', '仕入先', '株主構成','株主・代表取締役一致',
       '第一位株主名称','第一位株主_株式保有割合','第二位株主名称','第二位株主_株式保有割合','第三位株主名称','第三位株主_株式保有割合','取引先',
       '財務情報1_時期','財務情報1_売上(千円)', '財務情報1_純利益(千円)', '財務情報2_時期', '財務情報2_売上(千円)','財務情報2_純利益(千円)', '財務情報3_時期', '財務情報3_売上(千円)', '財務情報3_純利益(千円)',
       '取引銀行','売上伸長率', '利益伸長率', 'TSR備考', '代表者住所', '代表者生年月日', '出身地', '出身校','TSRデータ格納区分']]
df_update_all = pd.merge(sf_TSR_ID,df_update_all,on='企業コード(TSR)')
df_update_all.to_csv('df_update_all_2.csv',index = False)

df_update_all(3968件)："TSR企業コード"をキーに全項目アップデート


In [224]:
print(f'df_update合計({len(df_update_part) + len(df_update_all)})件')

df_update合計(4019)件


In [253]:
df_update_all.columns

Index(['企業情報 ID', '企業コード(TSR)', '上場/未上場', '上場区分', 'TSR調査年月日', '企業名ﾌﾘｶﾞﾅ（半角）',
       '代表者氏名ﾌﾘｶﾞﾅ（半角）', '会社名', '代表者氏名(姓)', '代表者氏名(名)', '郵便番号', '都道府県', '市区町村',
       '町名・番地', '建物名', '電話番号', '設立年月日', '創業年月日', '資本金（千円単位）', '従業員数', '大分類1',
       '中分類コード1', '中分類1', '小分類コード1', '小分類1', '細分類コード1', '細分類1', '大分類2',
       '中分類コード2', '中分類2', '小分類コード2', '小分類2', '細分類コード2', '細分類2', '大分類3',
       '中分類コード3', '中分類3', '小分類コード3', '小分類3', '細分類コード3', '細分類3', '事業概要',
       '営業所・支店住所', '役員', '仕入先', '株主構成', '株主・代表取締役一致', '第一位株主名称',
       '第一位株主_株式保有割合', '第二位株主名称', '第二位株主_株式保有割合', '第三位株主名称', '第三位株主_株式保有割合',
       '取引先', '財務情報1_時期', '財務情報1_売上(千円)', '財務情報1_純利益(千円)', '財務情報2_時期',
       '財務情報2_売上(千円)', '財務情報2_純利益(千円)', '財務情報3_時期', '財務情報3_売上(千円)',
       '財務情報3_純利益(千円)', '取引銀行', '売上伸長率', '利益伸長率', 'TSR備考', '代表者住所', '代表者生年月日',
       '出身地', '出身校', 'TSRデータ格納区分'],
      dtype='object')

In [254]:
df_update2 = df_merge3[~df_merge3['企業コード(TSR)'].isin(sf_TSR['企業コード(TSR)'])]
print(f'df_update2：{len(df_update2)}件')
df_update2.columns

df_update2：11件


Index(['企業コード(TSR)', '上場/未上場', '上場区分', 'TSR調査年月日', '企業名ﾌﾘｶﾞﾅ（半角）',
       '代表者氏名ﾌﾘｶﾞﾅ（半角）', '会社名', '代表者氏名', '代表者氏名(姓)', '代表者氏名(名)', '郵便番号',
       '都道府県', '市区町村', '町名・番地', '建物名', '電話番号', '設立年月日', '創業年月日', '資本金（千円単位）',
       '従業員数', '業種1No', '業種1', '業種2No', '業種2', '業種3No', '業種3', '事業概要',
       '営業所・支店住所', '役員', '仕入先', '株主構成', '株主・代表取締役一致', '第一位株主名称',
       '第一位株主_株式保有割合', '第二位株主名称', '第二位株主_株式保有割合', '第三位株主名称', '第三位株主_株式保有割合',
       '取引先', '財務情報1_時期', '財務情報1_売上(千円)', '財務情報1_純利益(千円)', '財務情報2_時期',
       '財務情報2_売上(千円)', '財務情報2_純利益(千円)', '財務情報3_時期', '財務情報3_売上(千円)',
       '財務情報3_純利益(千円)', '取引銀行', '売上伸長率', '利益伸長率', 'TSR備考', '代表者住所', '代表者生年月日',
       '出身地', '出身校', '代表者氏名チェック用', '大分類1', '中分類コード1', '中分類1', '小分類コード1',
       '小分類1', '細分類コード1', '細分類1', '大分類2', '中分類コード2', '中分類2', '小分類コード2', '小分類2',
       '細分類コード2', '細分類2', '大分類3', '中分類コード3', '中分類3', '小分類コード3', '小分類3',
       '細分類コード3', '細分類3'],
      dtype='object')

### （ⅲ）会社属性（会社名、代表者名、郵便番号）でマージ

In [255]:
df_update2_match = pd.merge(df_update2,sf_NoTSR,on=['会社名','代表者氏名','郵便番号'])
print(f'df_update2_attributes({len(df_update2_match)}件)：会社名、代表者名、郵便番号をキーに"TSR企業コードも"アップデート')
df_update2_match

df_update2_attributes(3件)：会社名、代表者名、郵便番号をキーに"TSR企業コードも"アップデート


,企業コード(TSR)_x,上場/未上場_x,上場区分,TSR調査年月日,企業名ﾌﾘｶﾞﾅ（半角）_x,代表者氏名ﾌﾘｶﾞﾅ（半角）_x,会社名,代表者氏名,代表者氏名(姓)_x,代表者氏名(名)_x,...,財務情報3_純利益(千円)※2021年下期TSR,取引銀行_y,売上伸長率※2021年下期,利益伸長率※2021年下期,TSR備考_y,代表者住所_y,代表者生年月日_y,出身地_y,出身校_y,最終活動日
0,300641818,グロース,上場,2022/06/30,ﾃｸﾉﾛｼﾞ-ｽﾞ,ﾖｼﾊﾗ ﾋﾛｷ,株式会社テクノロジーズ,良原 広樹,良原,広樹,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023/7/18
1,693816007,プライム,上場,2022/11/09,ｼﾞﾔﾊﾟﾝｸﾗﾌﾄﾎ-ﾙﾃﾞｲﾝｸﾞｽ,ﾅｶﾏﾂ ｹﾝｲﾁ,ジャパンクラフトホールディングス株式会社,中松 健一,中松,健一,...,NaN,名古屋銀行／愛知銀行／北陸銀行／商工組合中央金庫／十六銀行／三菱ＵＦＪ銀行／三菱ＵＦＪ信託銀...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,692795812,プライム,上場,2022/08/29,ﾔﾏｴｸﾞﾙ-ﾌﾟﾎ-ﾙﾃﾞｲﾝｸﾞｽ,ｵｳﾀﾞ ﾋﾃﾞﾄ,ヤマエグループホールディングス株式会社,網田 日出人,網田,日出人,...,NaN,NaN,NaN,NaN,ヤマエ久野株式会社のホールディングス会社,NaN,NaN,NaN,NaN,2023/7/31


In [256]:
df_update2_Nomatch = df_update2[~df_update2['企業コード(TSR)'].isin(df_update2_match['企業コード(TSR)_x'])]
df_update2_Nomatch

,企業コード(TSR),上場/未上場,上場区分,TSR調査年月日,企業名ﾌﾘｶﾞﾅ（半角）,代表者氏名ﾌﾘｶﾞﾅ（半角）,会社名,代表者氏名,代表者氏名(姓),代表者氏名(名),...,小分類2,細分類コード2,細分類2,大分類3,中分類コード3,中分類3,小分類コード3,小分類3,細分類コード3,細分類3
341,452117186,グロース,上場,2023/04/18,ｱﾚﾝﾄ,ｶﾓﾊﾞﾔｼ ﾋﾛｷ,株式会社Ａｒｅｎｔ,鴨林 広軌,鴨林,広軌,...,ソフトウェア業,3911,受託開発ソフトウェア業,情報通信業,39,情報サービス業,391,ソフトウェア業,3912,組込みソフトウェア業
357,14774100,グロース,上場,2023/01/20,ﾄﾘﾌﾟﾗ,ﾀｶﾊｼ ｶｽﾞﾋｻ,ｔｒｉｐｌａ株式会社,髙橋 和久,髙橋,和久,...,ソフトウェア業,3912,組込みソフトウェア業,情報通信業,39,情報サービス業,392,情報処理・提供サービス業,3922,情報提供サービス業
407,298883163,グロース,上場,2023/01/30,ﾉ-ﾄ,ｶﾄｳ ｻﾀﾞｱｷ,ｎｏｔｅ株式会社,加藤 貞顕,加藤,貞顕,...,インターネット附随サービス業,4012,ＡＳＰ・ウェブコンテンツ提供業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,133439488,グロース,上場,2022/05/24,ｴﾆ-ﾏｲﾝﾄﾞｸﾞﾙ-ﾌﾟ,ｿｺﾞｳ ｺｳｽｹ,ＡｎｙＭｉｎｄ Ｇｒｏｕｐ株式会社,十河 宏輔,十河,宏輔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558,300506082,グロース,上場,2023/01/24,ﾘﾍﾞ-ｽ,ｻﾄｳ ｶｲ,株式会社Ｒｅｂａｓｅ,佐藤 海,佐藤,海,...,他に分類されない事業サービス業,9299,その他の事業サービス業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
653,296296945,グロース,上場,2022/12/08,ﾃｲﾑｽ,ﾜｶﾊﾞﾔｼ ﾀｸﾛｳ,株式会社ティムス,若林 拓朗,若林,拓朗,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,295270144,スタンダ,上場,2022/04/28,ｺ-ﾁｴｲ,ｽｽﾞｷ ﾖｼﾕｷ,株式会社コーチ・エィ,鈴木 義幸,鈴木,義幸,...,その他の専門サービス業,7299,他に分類されない専門サービス業,教育，学習支援業,82,その他の教育，学習支援業,822,職業・教育支援施設,8221,職員教育施設・支援業
1144,32447760,スタンダ,上場,2022/07/22,ｴｷｻｲﾄﾎ-ﾙﾃﾞｲﾝｸﾞｽ,ｻｲｼﾞﾖｳ ｼﾝｲﾁ,エキサイトホールディングス株式会社,西條 晋一,西條,晋一,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [257]:
#会社名以外が変更
sf_change_ExeCompany = sf_change[~sf_change['変更内容選択（事務記載）'].str.contains('会社名変更')]
df_update2_merge = pd.merge(sf_change_ExeCompany,df_update2_Nomatch,on = ['会社名','代表者氏名'])
df_update2_merge

,企業情報 ID,企業コード(TSR)_x,活動 ID,件名,会社名,作成日,代表者氏名,代表者氏名ﾌﾘｶﾞﾅ（半角）_x,郵便番号_x,電話,...,小分類2,細分類コード2,細分類2,大分類3,中分類コード3,中分類3,小分類コード3,小分類3,細分類コード3,細分類3


In [258]:
df_insert = df_update2_Nomatch[~df_update2_Nomatch['会社名'].isin(df_update2_merge['会社名'])]
df_insert

,企業コード(TSR),上場/未上場,上場区分,TSR調査年月日,企業名ﾌﾘｶﾞﾅ（半角）,代表者氏名ﾌﾘｶﾞﾅ（半角）,会社名,代表者氏名,代表者氏名(姓),代表者氏名(名),...,小分類2,細分類コード2,細分類2,大分類3,中分類コード3,中分類3,小分類コード3,小分類3,細分類コード3,細分類3
341,452117186,グロース,上場,2023/04/18,ｱﾚﾝﾄ,ｶﾓﾊﾞﾔｼ ﾋﾛｷ,株式会社Ａｒｅｎｔ,鴨林 広軌,鴨林,広軌,...,ソフトウェア業,3911,受託開発ソフトウェア業,情報通信業,39,情報サービス業,391,ソフトウェア業,3912,組込みソフトウェア業
357,14774100,グロース,上場,2023/01/20,ﾄﾘﾌﾟﾗ,ﾀｶﾊｼ ｶｽﾞﾋｻ,ｔｒｉｐｌａ株式会社,髙橋 和久,髙橋,和久,...,ソフトウェア業,3912,組込みソフトウェア業,情報通信業,39,情報サービス業,392,情報処理・提供サービス業,3922,情報提供サービス業
407,298883163,グロース,上場,2023/01/30,ﾉ-ﾄ,ｶﾄｳ ｻﾀﾞｱｷ,ｎｏｔｅ株式会社,加藤 貞顕,加藤,貞顕,...,インターネット附随サービス業,4012,ＡＳＰ・ウェブコンテンツ提供業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,133439488,グロース,上場,2022/05/24,ｴﾆ-ﾏｲﾝﾄﾞｸﾞﾙ-ﾌﾟ,ｿｺﾞｳ ｺｳｽｹ,ＡｎｙＭｉｎｄ Ｇｒｏｕｐ株式会社,十河 宏輔,十河,宏輔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558,300506082,グロース,上場,2023/01/24,ﾘﾍﾞ-ｽ,ｻﾄｳ ｶｲ,株式会社Ｒｅｂａｓｅ,佐藤 海,佐藤,海,...,他に分類されない事業サービス業,9299,その他の事業サービス業,NaN,NaN,NaN,NaN,NaN,NaN,NaN
653,296296945,グロース,上場,2022/12/08,ﾃｲﾑｽ,ﾜｶﾊﾞﾔｼ ﾀｸﾛｳ,株式会社ティムス,若林 拓朗,若林,拓朗,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,295270144,スタンダ,上場,2022/04/28,ｺ-ﾁｴｲ,ｽｽﾞｷ ﾖｼﾕｷ,株式会社コーチ・エィ,鈴木 義幸,鈴木,義幸,...,その他の専門サービス業,7299,他に分類されない専門サービス業,教育，学習支援業,82,その他の教育，学習支援業,822,職業・教育支援施設,8221,職員教育施設・支援業
1144,32447760,スタンダ,上場,2022/07/22,ｴｷｻｲﾄﾎ-ﾙﾃﾞｲﾝｸﾞｽ,ｻｲｼﾞﾖｳ ｼﾝｲﾁ,エキサイトホールディングス株式会社,西條 晋一,西條,晋一,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
#インサートの場合は所有者を山口さんに指定
df_insert['ownerid'] = '0055j0000077gFcAAI'
df_insert['TSRデータ格納区分'] = '2023年上期インサート'
df_insert = df_insert[['企業コード(TSR)', '上場/未上場','上場区分', 'TSR調査年月日', '企業名ﾌﾘｶﾞﾅ（半角）', '代表者氏名ﾌﾘｶﾞﾅ（半角）',
       '会社名', '代表者氏名(姓)', '代表者氏名(名)', '郵便番号', '都道府県', '市区町村', '町名・番地',
       '建物名', '電話番号', '設立年月日', '創業年月日', '資本金（千円単位）', '従業員数',
       '大分類1', '中分類コード1', '中分類1', '小分類コード1','小分類1', '細分類コード1', '細分類1',
       '大分類2', '中分類コード2', '中分類2', '小分類コード2', '小分類2','細分類コード2', '細分類2',
       '大分類3', '中分類コード3', '中分類3', '小分類コード3', '小分類3','細分類コード3', '細分類3', 
       '事業概要', '営業所・支店住所', '役員', '仕入先', '株主構成','株主・代表取締役一致',
       '第一位株主名称','第一位株主_株式保有割合','第二位株主名称','第二位株主_株式保有割合','第三位株主名称','第三位株主_株式保有割合','取引先',
       '財務情報1_時期','財務情報1_売上(千円)', '財務情報1_純利益(千円)', '財務情報2_時期', '財務情報2_売上(千円)','財務情報2_純利益(千円)', '財務情報3_時期', '財務情報3_売上(千円)', '財務情報3_純利益(千円)',
       '取引銀行','売上伸長率', '利益伸長率', 'TSR備考', '代表者住所', '代表者生年月日', '出身地', '出身校','TSRデータ格納区分','ownerid']]
df_insert.to_csv('df_insert.csv')

In [260]:
os.getcwd()

'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\TSRデータ更新\\2023年上期\\2023年上期_TSR元データ\\上場\\上場'

In [160]:
os.chdir('C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\2022年下期\\2022年下期_Salesforce格納用データ\\2022下期_インサートファイル')
df_completed = pd.read_csv('2022年下期インサート.csv',encoding='cp932')
df_completed['企業コード(TSR)'] = df_completed['企業コード(TSR)'].fillna(0)
df_completed['企業コード(TSR)'] = df_completed['企業コード(TSR)'].astype(int)
df_completed

,企業情報 ID,企業コード(TSR),会社名,TSRデータ格納区分
0,0015j00000dFyEX,25242989,株式会社ペイミー,2022年下期インサート
1,0015j00000dG4sD,670137561,兵庫シューズ株式会社,2022年下期インサート
2,0015j00000dFyEV,14278332,株式会社フォレストアドベンチャー,2022年下期インサート
3,0015j00000dG4sE,675243343,株式会社水宝,2022年下期インサート
4,0015j00000dFyEW,291715460,不二ビル管理株式会社,2022年下期インサート
...,...,...,...,...
47323,0015j00000dFvaU,131494430,株式会社久野林業,2022年下期インサート
47324,0015j00000dFvaV,190102730,有限会社佐々直建築,2022年下期インサート
47325,0015j00000dFvaW,192012525,新産測量株式会社,2022年下期インサート
47326,0015j00000dFva5,180170090,山武石油株式会社,2022年下期インサート


In [161]:
df_addinsert = df_insert[~df_insert['企業コード(TSR)'].isin(df_completed['企業コード(TSR)'])]
df_addinsert.to_csv('追加インサート分.csv',encoding = 'cp932')

### （ⅳ）インサート＆アップデート後の修正

In [131]:
df_ku =pd.read_csv('町名に区を含むもの.csv',encoding = 'cp932')

In [134]:
df_ku_TSR = df_ku[~df_ku['企業コード(TSR)'].isnull()]
df_ku_notTSR = df_ku[df_ku['企業コード(TSR)'].isnull()]
df_ku_notTSR

,企業情報ID,企業コード(TSR),会社名,都道府県,市区町村,町名・番地
0,0015j00000yoJ4p,NaN,安田金属ホールディングス,大阪府,大阪市,中央区南船場１－１１－９
55,0015j00000rrrUo,NaN,ＦＵＥＬ株式会社,東京都,港区,東京都港区虎ノ門４－１－１４
576,0015j00000yoJgP,NaN,株式会社エフ・クラウド,大阪府,大阪市,北区梅田１－３－１－１０００
20604,0015j00000mKWV1,NaN,塩谷工業株式会社,兵庫県,姫路市,飾磨区恵美酒２５０－５
32477,0015j00000xVFEO,NaN,医療法人藤慈会,埼玉県,さいたま市,大宮区堀の内町２―５６４
...,...,...,...,...,...,...
133825,0015j00000xVF1m,NaN,医療法人社団仁明会,神奈川県,横浜市,港南区上永谷２―１０―３６
133826,0015j00000xVF1q,NaN,医療法人社団鴨居病院,神奈川県,横浜市,緑区鴨居５―２７―１０
133827,0015j00000xVF1s,NaN,医療法人社団同仁会,神奈川県,横浜市,都筑区中川４―２―２０
133828,0015j00000xVF1t,NaN,医療法人社団秀仁会,神奈川県,横浜市,西区楠町５―１


In [136]:
df_ku_TSR['企業コード(TSR)'] = df_ku_TSR['企業コード(TSR)'].astype(int)

In [146]:
df_ku_TSR_tatemono = pd.merge(df_ku_TSR,df_,on='企業コード(TSR)')
df_ku_TSR_tatemono.to_csv('区を修正.csv')

In [143]:
df_ku_TSR['所在地'] = df_ku_TSR['都道府県']+df_ku_TSR['市区町村']+df_ku_TSR['町名・番地']+df_ku_TSR['建物名']
df_ku_notTSR['所在地'] = df_ku_notTSR['都道府県']+df_ku_notTSR['市区町村']+df_ku_notTSR['町名・番地']

KeyError: '建物名'

In [141]:
match['所在地']
df_ku_notTSR = df_ku_notTSR[~df_ku_notTSR['所在地'].isnull()]
df_ku_TSR

,企業情報ID,企業コード(TSR),会社名,都道府県,市区町村,町名・番地
1,0015j00000f4gei,10367101,株式会社ノーザンクロス,北海道,札幌市,中央区大通東２-３-１
2,0015j00000fpw5I,14611031,株式会社ＴＲＡＳＴＹ,大阪府,大阪市,東淀川区大道南１-３-１３-１Ｆ
3,0015j00000fqHrI,352340355,株式会社テルトル・ルージュ,神奈川県,横浜市,港南区日野５-２３-２１
4,0015j00000fq0t9,571661807,株式会社キタイコーポレーション,大阪府,大阪市,鶴見区今津北４-１０-１
5,0015j00000fqJ0u,350531455,株式会社坂井工務店,神奈川県,横浜市,鶴見区下野谷町４-１３９
...,...,...,...,...,...,...
133936,0015j00000dFvl2,142321540,株式会社ＨＯＷ’Ｓ,宮城県,仙台市,青葉区一番町４-１-１
133937,0015j00000dFvl3,142331465,株式会社スタークス,宮城県,仙台市,青葉区五橋１-１-１０
133938,0015j00000dFvl8,132243598,シンエイ株式会社,宮城県,仙台市,青葉区大町１-２-１
133939,0015j00000dFvkh,18132154,仙台スマートマシーンズ株式会社,宮城県,仙台市,青葉区荒巻字青葉６-６-４０


In [122]:
df_ku_notTSR['所在地']

0           大阪府大阪市中央区南船場１－１１－９
55         東京都港区東京都港区虎ノ門４－１－１４
576       大阪府大阪市北区梅田１－３－１－１０００
20604        兵庫県姫路市飾磨区恵美酒２５０－５
32477     埼玉県さいたま市大宮区堀の内町２―５６４
                  ...         
133825    神奈川県横浜市港南区上永谷２―１０―３６
133826      神奈川県横浜市緑区鴨居５―２７―１０
133827      神奈川県横浜市都筑区中川４―２―２０
133828          神奈川県横浜市西区楠町５―１
133834     兵庫県神戸市垂水区塩屋町９－２６－１７
Name: 所在地, Length: 1046, dtype: object

In [109]:
match['都道府県'] = match['所在地'].apply(lambda x:addres_split(x)[0])
df_ku_notTSR['都道府県'] = df_ku_notTSR['所在地'].apply(lambda x:addres_split(x)[0])
df_ku_notTSR['都道府県']

Series([], Name: 都道府県, dtype: object)

In [110]:
match['都道府県'] = match['所在地'].apply(lambda x:addres_split(x)[0])
match['市区町村'] = match['所在地'].apply(lambda x:addres_split(x)[1])
match['町名・番地'] = match['所在地'].apply(lambda x:addres_split(x)[2])
# match['町名・番地'] = match['町名・番地'].str.replace('－','-')
match['建物名'] = match['所在地'].apply(lambda x:addres_split(x)[3])
# match['建物名'] = match['建物名'].str.replace('－','-')

In [123]:
df_ku_notTSR['都道府県'] = df_ku_notTSR['所在地'].apply(lambda x:addres_split(x)[0])
df_ku_notTSR['市区町村'] = df_ku_notTSR['所在地'].apply(lambda x:addres_split(x)[1])
df_ku_notTSR['町名・番地'] = df_ku_notTSR['所在地'].apply(lambda x:addres_split(x)[2])
# df_ku_notTSR['町名・番地'] = df_ku_notTSR['町名・番地'].str.replace('－','-')
df_ku_notTSR['建物名'] = df_ku_notTSR['所在地'].apply(lambda x:addres_split(x)[3])
# df_ku_notTSR['建物名'] = df_ku_notTSR['建物名'].str.replace('－','-')

In [124]:
match['都道府県']
df_ku_notTSR['都道府県']

0          大阪府
55         東京都
576        大阪府
20604      兵庫県
32477      埼玉県
          ... 
133825    神奈川県
133826    神奈川県
133827    神奈川県
133828    神奈川県
133834     兵庫県
Name: 都道府県, Length: 1046, dtype: object

In [125]:
match['市区町村']
df_ku_notTSR['市区町村']

0           大阪市中央区
55              港区
576          大阪市北区
20604       姫路市飾磨区
32477     さいたま市大宮区
            ...   
133825      横浜市港南区
133826       横浜市緑区
133827      横浜市都筑区
133828       横浜市西区
133834      神戸市垂水区
Name: 市区町村, Length: 1046, dtype: object

In [126]:
match['町名・番地']
df_ku_notTSR['町名・番地']

0              南船場１－１１－９
55        東京都港区虎ノ門４－１－１４
576         梅田１－３－１－１０００
20604           恵美酒２５０－５
32477          堀の内町２―５６４
               ...      
133825        上永谷２―１０―３６
133826         鴨居５―２７―１０
133827          中川４―２―２０
133828             楠町５―１
133834        塩屋町９－２６－１７
Name: 町名・番地, Length: 1046, dtype: object

In [130]:
df_ku.to_csv('区を修正.csv',encoding='cp932')
df_ku_notTSR.to_csv('区を修正2.csv',encoding='cp932')

PermissionError: [Errno 13] Permission denied: '区を修正.csv'

In [69]:
os.getcwd()

'C:\\Users\\miyab\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\2022年下期\\2022年下期_Salesforce格納用データ\\2022下期_アップデートファイル'

In [115]:
df_ku_notTSR

,企業情報ID,企業コード(TSR),会社名,都道府県,市区町村,町名・番地,所在地,建物名
